# Extracting Movie Titles with Hidden Markov Models

English sentences follow a similar structure when it comes to referring to a named entity.  There is a lot of success in named entity recognition when extracting locations, times, and people from text.  Less successful, however, is when we are dealing with names that are synonymous with common words.  Even moreso when the data we are dealing with informal data, such as tweets from Twitter.  This experiment aims to extract movie titles from Twitter data using Hidden Markov Models (HMMs).  The main idea is that while every tweet is unique, a common sentence structure (part-of-speech tags) sequence can be learned from all of the tweets.

We will be using HMMs to build a language model from our training data.  Our training data consists of tweets that mention a movie, and include other tweets that do not mention a movie, but may contain a word that is synonymous with a movie.  For example,

* " did you do your homework yet ? " " no I 'm watching MovieTitle ( Frozen ) "
* So my feet are frozen .... I leave the windows open , 24/7
* Genuinely love MovieTitle ( frozen ) & lt ; 3
* MovieTitle ( FROZEN ) IN 3 D IS THE BEST THING EVER

After creating our language model, we will test it using two separate datasets, each with their own gazetteers.  The first test dataset contains movies that are the same as those in our training dataset, while the second dataset contains movies that are not the same as those in our training dataset.  Both datasets may contain tweets that do not refer a movie, but still contain a word that is synonymous with a movie.

Small sample of tweets from test dataset 1:
* my face is pretty much frozen
* Finally getting to watch MovieTitle ( Frozen ) with Kynnedi ❄️
* About to go watch MovieTitle ( Frozen ) .
* the kids im babysitting are listening to music from MovieTitle ( frozen ) #ok

Small sample of tweets from test dataset 2:
* Ok .... Matthew did the damn thing in MovieTitle ( Dallas Buyers Club )
* This is funnier than her crying at MovieTitle ( the LEGO movie )
* RT @birbigs : MovieTitle ( Dallas Buyers Club ) is one of the rare movies that lives up to the hype . If you have n't already , SEE IT .
* RT @loadedmag RT 2 Win ! In cinemas today , MovieTitle ( Lone survivor ) , four men face an army , To celebrate were giving away KINDLE FIRE ! #LOSUUK


Training and test data was obtained from: https://github.com/sandeepAshwini/TwitterMovieData

Movie titles can be downloaded from http://www.imdb.com/interfaces

## Preliminary Modules

Before we begin, let's get these import statements out of the way.

In [ ]:
import nltk
from collections import defaultdict

We use nltk for part-of-speech tagging, and defaultdict for maintaining a dictionary.

## Data Normalization

First and foremost, we notice that there is a lot of noise in each tweet that does not contribute to the reference of a movie.  For example, hyperlinks, double quotation marks, and "RT" does not help decipher whether a user is referring to a movie or not.  Because of this, we need to normalize the tweets.

In [ ]:
def normalize(line):
    tokenized_line = line.split()
    new_list = []
    for token in tokenized_line:
        if not any(x in token for x in ["@", "\"", "http://", "RT"]):
            new_list.append(token)
        if "@" in token:
            new_list.append("Cody")
    return " ".join(new_list)

As the above code describes, we go through the entire line, removing anything that's a mention/reply, double quote, hyperlink, and RT.  We also make sure to be very careful about mentions.  Instead of removing them, we note that they actually contain useful information regarding the context of the tweet, so we substitute it for an actual proper noun instead.  I substituted it for my name, but I'm sure a unique placeholder would be a better idea.

Next, in order to build a language model from our training data, we need to replace the substring "..MovieTitle ( * ).." in the tweet with a placeholder string.  In this case, we use the substring "#MOVIE#".

In [ ]:
def switch_title_with_placeholder(line):
    before_title = line[:line.find("MovieTitle (") - 1]
    including_title = line[line.find("MovieTitle ("):]
    after_title = including_title[including_title.find(")") + 2:]
    if line.find("MovieTitle (") == 0:
        return " ".join(["#MOVIE#", after_title])
    else:
        return " ".join([before_title, "#MOVIE#", after_title])

We are now ready to train our language model.

## Training the Language Model

As a Hidden Markov Model uses initial states, transitions, and emissions from states, we need to define these as well as find their probabilities.  We will let the states be defined as the word's part-of-speech tag, and the emission will be defined as the word itself.  

In [ ]:
def get_and_store_pos_tag(line):
    splitted_line = line.split()
    pos_tags = nltk.pos_tag(splitted_line)
    for item in pos_tags:
        if item[0] == "#MOVIE#":
            current_word = "#MOVIE#"
            current_tag = "#MOVIE#"
        else:
            current_word = item[0]
            current_tag = item[1]
        if pos_tags.index(item) == 0:
            initial_state_counts[current_tag] += 1
        elif pos_tags.index(item) >= 1:
            if state_transition_counts[previous_tag] == False:
                state_transition_counts[previous_tag] = defaultdict(lambda: 0.001)
            state_transition_counts[previous_tag][current_tag] += 1
            dest_src_transitions[current_tag].append(previous_tag)
        if state_output_counts[current_tag] == False:
            state_output_counts[current_tag] = defaultdict(lambda: 0.001)
        state_output_counts[current_tag][current_word] += 1
        outputs_to_states[current_word].append(current_tag)
        previous_tag = current_tag

The function above determines a tweet's part-of-speech tags and stores the number of times:
* That state (ie. part-of-speech tag) is at the beginning of the tweet
* We transition from one state to another
* The word is emitted from that state

Some states may not transition from one state to another in our training data yet arise in our test data, so we assign those occurences with a probability of 0.001.  We do the same thing for our emission probabilities (ie. if we come across an emission that we have never seen in our training data, we assign it a small probability of 0.001).

Also note the variables dest_src_transitions and outputs_to_states.  These variables make it easier and quicker to do a lookup of the states and outputs later on when we need to calculate the probability of a certain sequence.
* dest_src_transitions is a dictionary where key=destination state, and value=list of states that transition to that destination state
* outputs_to_states is a dictionary where key=emitted word, and value=list of states that emit that word.

Next, we want to convert those counts to actual probabilities.

In [ ]:
def convert_counts_to_probs():
    total_initial_states = 0
    for item in initial_state_counts.iterkeys():
        total_initial_states += initial_state_counts[item]
    for item in initial_state_counts.iterkeys():
        initial_state_counts[item] = float(initial_state_counts[item])/float(total_initial_states)

    for start_key in state_transition_counts.iterkeys():
        total_transition_counts = 0
        for end_key in state_transition_counts[start_key]:
            total_transition_counts += state_transition_counts[start_key][end_key]
        for end_key in state_transition_counts[start_key]:
            state_transition_counts[start_key][end_key] = float(state_transition_counts[start_key][end_key]) / float(
                total_transition_counts)

    for state in state_output_counts.iterkeys():
        total_output_counts = 0
        for output in state_output_counts[state]:
            total_output_counts += state_output_counts[state][output]
        for output in state_output_counts[state]:
            state_output_counts[state][output] = float(state_output_counts[state][output]) / float(
                total_output_counts)

We can now pipe the functions above together to create our language model from the training data.

In [ ]:
def train(train_data):
    with open(train_data) as train_file:
        for line in train_file.readlines():
            normalized_line = normalize(line)
            while "MovieTitle (" in normalized_line:
                normalized_line = switch_title_with_placeholder(normalized_line)
            get_and_store_pos_tag(normalized_line)
        convert_counts_to_probs()

The above function trains our language model over all tweets in our training data.  Some tweets refer to more than one movie, so we iterate over the tweet as many times as necessary in order to replace all movie titles with placeholder text.

## The Forward Algorithm

To build our language model, we will use the Forward algorithm.  This algorithm uses dynamic programming (stores results in a table in order to avoid recalculations) in order to calculate the probability of a sequence of emissions.  Given table v, state q, and index t, the forward algorithm is as follows:
~~~
for each state q that emits the word at index 1
    v[q, 1] = Pr(initial state=q) * Pr(state=q, emission=word at index 1)
from t = 2 to end of tweet
    v[q, t] += v[q', t-1] * Pr(src state=q', dest state=q) * Pr(current state=q, emission=word at index t)
~~~
Our implementation will follow the same algorithm, but will also incorporate some smoothing in areas where the Forward Algorithm is too harsh.

In [ ]:
def forward_algorithm(line):
    splitted_line = line.split()
    v = defaultdict(float)
    probability = 0.001
    for item in set(outputs_to_states[splitted_line[0]]):
        v[(item, 0)] = initial_state_counts[item] * state_output_counts[item][splitted_line[0]]
    for x in range(1, len(splitted_line)):
        for item_dest in set(outputs_to_states[splitted_line[x]]):
            for item in set(outputs_to_states[splitted_line[x-1]]):
                if v[(item, x-1)] == 0.0:
                    v[(item, x - 1)] = probability * 0.0000001
                v[(item_dest, x)] += (v[(item, x-1)] * state_transition_counts[item][item_dest] * state_output_counts[item_dest][splitted_line[x]])
                probability = v[(item_dest, x)]
    return probability

Problems in the original Forward algorithm occur when there is no state in that emits a word.  Since there is no state that emits it, our outputs_to_states value for that emission key would be empty, thus causing us to move onto the next iteration of the loop.  This puts 0 in those table slots, which ruins our algorithm.  Our workaround is the following: If there is no state that emits a specific state that we are looking up, assign it as the old probability * a very very low probability.  This causes our algorithm to favor tweets that contain words and part-of-speech tags that we are more familiar with.

We are now ready to evaluate our test data.

## Evaluating Test Data

Now that we have defined the Forward algorithm, we can now attempt to extract movie titles from them.  To do this we need to normalize the text, extract the original tweet from the "..MovieTitle ( * ).." tagged tweet, generate candidate movies by comparing the words in the sentence with the movies in our gazetteer, and then perform the Forward algorithm on each candidate.  Candidates that score higher than the original tweet are considered as movies.

In [ ]:
def get_original_sentence(sentence_number, line):
    before_title = line[:line.find("MovieTitle (") - 1]
    including_title = line[line.find("MovieTitle ("):]
    movie_title = including_title[len("MovieTitle ("):including_title.find(")")]
    after_title = including_title[including_title.find(")") + 2:]
    gazetteer.add(movie_title[1:-1])
    sentence_to_movies[sentence_number].append(movie_title[1:-1])
    if line.find("MovieTitle (") == 0:
        return movie_title[1:] + after_title
    else:
        return before_title + movie_title + after_title

This function is very similar to `switch_title_with_placeholder(line)`, the difference being that it removes the MovieTitle tags, and the movie to a gazetteer for searching through later.

We can now pipe our functons together to evaluate our test data.

In [ ]:
def test(test_data):
    global true_positive
    global false_negative
    global false_positive
    with open(test_data) as test_file:
        original_sentences = []
        for (i, line) in enumerate(test_file.readlines()):
            normalized_line = normalize(line)
            while "MovieTitle (" in normalized_line:
                normalized_line = get_original_sentence(i, normalized_line)
            original_sentences.append(normalized_line)
        for sentence in original_sentences:
            potential_movies = []
            print sentence
            original_prob = forward_algorithm(sentence)
            candidate_indices = []
            splitted_sentence = sentence.split()
            for x in range(0, len(splitted_sentence)):
                for y in range(x+1, len(splitted_sentence)+1):
                    if " ".join(splitted_sentence[x:y]) in gazetteer:
                        candidate_indices.append([x, y])
            for candidate_indice in candidate_indices:
                splitted_sentence = sentence.split()
                movie_title = " ".join(splitted_sentence[candidate_indice[0]:candidate_indice[1]])
                splitted_sentence[candidate_indice[1] - 1] = "#MOVIE#"
                del splitted_sentence[candidate_indice[0]:candidate_indice[1] - 1]
                candidate_sentence = " ".join(splitted_sentence)
                candidate_prob = forward_algorithm(candidate_sentence)
                if candidate_prob > original_prob:
                    potential_movies.append(movie_title)

            actual = set(sentence_to_movies[original_sentences.index(sentence)])
            candidates = set(potential_movies)
            print "Actual:", ", ".join(actual)
            print "Guesses:", ", ".join(candidates)
            local_true_positive = len(actual.intersection(candidates))
            if len(actual) == 0 and len(candidates) == 0:
                true_positive += 1
            local_false_negative = len(actual - candidates)
            local_false_positive = len(candidates - actual)

            true_positive += local_true_positive
            false_negative += local_false_negative
            false_positive += local_false_positive

The variables at the end of the function (ie. true positive, false negative, false positive) are used for quantifying the quality of our algorithm.  True positive refers to movie titles that our algorithm thinks is correct and is correct (based on the tagged movies in that tweet).  False negative refers to movie titles that our algorithm dismissed but is correct.  False positive refers to movie titles that our algorithm thinks is correct but is not correct.

## Putting It All Together

In [ ]:
initial_state_counts = defaultdict(float)
state_transition_counts = defaultdict(bool)
state_output_counts = defaultdict(bool)
outputs_to_states = defaultdict(list)
dest_src_transitions = defaultdict(list)

gazetteer = set()
with open("movies.list") as movies_list:
    for index, line in enumerate(movies_list):
        if index >= 16 and line[0] != "\"":
            gazetteer.add(line.split("(")[0])

true_positive = 0
false_negative = 0
false_positive = 0

train('TrainFile.labeled')
sentence_to_movies = defaultdict(list)
test('TestFile1.labeled')

precision = float(true_positive)/float(true_positive + false_positive)
recall = float(true_positive)/float(true_positive + false_negative)

print "True Positive =", true_positive
print "False Negative =", false_negative
print "False Positive =", false_positive
print "Precision =", precision
print "Recall =", recall
print "F-measure =", float(2 * precision * recall)/float(precision + recall)

`movies.list` was downloaded from the IMDB interfaces.  It contains a list of movies that we can use to compare with.

## Results

### Results of Test Dataset 1

Condition | Result
-------------|-------------
True Positive | 138
False Negative | 9
False Positive | 67
Precision | 0.673170731707
Recall | 0.938775510204
F-measure | 0.784090909091

### Results of Test Dataset 2

Condition | Result
-------------|-------------
True Positive | 128
False Negative | 36
False Positive | 22
Precision | 0.853333333333
Recall | 0.780487804878
F-measure | 0.815286624204


But what do these conditions mean?  Precision tells us the ratio of correctly guessed movies to correctly guessed movies and incorrectly guessed movies.  Recall tells us the ratio of correctly guessed movies to correctly guessed movies and incorrectly ignored movies.  F-measure is calculated by combining the precision and recall.  A high recall and low precision would yield the same F-measure as a high precision and a low recall would.

This means that in test dataset 1 our algorithm thinks less correct movies are incorrect, but thinks too many incorrect movies are correct.  The result is different for test dataset 2: it thinks less incorrect movies are correct, but thinks too many correct movies are incorrect.

In this scenario, it's difficult to know which of higher precision or higher recall is better.  We can obtain a higher recall score by accepting all candidate movies in our `test(test_data)` function.  This means that all candidate movies will be returned as "guessed" movies which brings our recall score much higher, but at the same time it means that it will return *every* movie that it sees, as long as the movie is in the gazetteer.  This isn't very wise.

With that said, it does a pretty good job of actually extracting movies from the tweets as well as ignoring words that are synonymous with movie titles.  For example,

* **Normalized tweet:** #OhHeyGuys would you rather I tweet #AmericanHustle or #Hobbit tomorrow ?
* **Actual movies:** #Hobbit, #AmericanHustle
* **Guessed movies:** #Hobbit, #AmericanHustle


* **Normalized tweet:** Cody seeing hobbit tonight ! Very excited . #bilbo #Hobbit
* **Actual:** hobbit
* **Guesses:** hobbit


* **Normalized tweet:** Me and a giant #Hobbit #dwarf #oxymoron at Auckland International Airport . See you shortly ...
* **Actual movie**: 
* **Guessed movie:**

The first example shows that our algorithm correctly extracts two movie titles from the tweet.  The second example shows that our algorithm is picks out the correct movie reference ("hobbit" and "#Hobbit" are in the gazetteer).  The third example shows that it doesn't consider #Hobbit to be a movie title.  While it is a movie title, the tweet refers to #Hobbit as a character, not the movie.

The biggest drawback to our algorithm is that it only finds movies that are in the gazetteer (ie. list of movie titles).  This means that without the gazetteer, it will not find any movies.  This is a problem because we would need to keep an updated list of movies and their varying reference phrases (e.g. Lord of the Rings, LOTR, LotR).  For example,

* **Normalized tweet:** The Lego movie was hilarious ! Sweet too ! And Nandos was gorgeous ! #twopuddings Cody ;)
* **Actual movie:** The Lego movie
* **Guessed movies:** Lego movie, The Lego movie

"Lego movie" and "The Lego movie" refer to the same movie, yet the program separates them because the two are separate entries in the gazetteer.  This is not optimal.  A post-processing step such as a gazetteer entry lookup function would solve this, but as stated before, requires an up-to-date list of variations of movie titles.